In [71]:
%load_ext autoreload
%autoreload 2
import hippo
import mrich
from mrich import print
from pathlib import Path
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
animal = hippo.HIPPO("RdRp", "../../BulkDock/TARGETS/Flavi_NS5_RdRp/Flavi_NS5_RdRp.sqlite")

 Creating HIPPO animal

name = RdRp

db_path = ../../BulkDock/TARGETS/Flavi_NS5_RdRp/Flavi_NS5_RdRp.sqlite

DEBUG: hippo.Database.__init__()

DEBUG: Database.path = ../../BulkDock/TARGETS/Flavi_NS5_RdRp/Flavi_NS5_RdRp.sqlite

DEBUG: hippo.Database.connect()

DEBUG: sqlite3.version='2.6.0'

 Success  Database connected @ ../../BulkDock/TARGETS/Flavi_NS5_RdRp/Flavi_NS5_RdRp.sqlite!

 Success  Initialised animal HIPPO("RdRp")!

## Export SMILES by pockets

In [20]:
TAGS = [
    "Green Site/RNA tunnel",
    "Primer grip/N-pocket",
    "Cavity B",
    "Thumb Site 2",
]

In [22]:
POSES = {}
for tag in TAGS:
    poses = animal.poses(tag=f"{tag} merge")
    print(poses)
    POSES[tag] = dict(merge_poses=poses)

poses tagged "Green Site/RNA tunnel merge": {P × 172966}

poses tagged "Primer grip/N-pocket merge": {P × 28894}

poses tagged "Cavity B merge": {P × 452}

poses tagged "Thumb Site 2 merge": {P × 4892}

In [27]:
%%time
for k,v in POSES.items():
    print(k)
    poses = v["merge_poses"]
    compounds = poses.compounds
    POSES[k]["merge_compounds"] = compounds
    POSES[k]["merge_smiles"] = compounds.smiles

Green Site/RNA tunnel

Primer grip/N-pocket

Cavity B

Thumb Site 2

CPU times: user 390 ms, sys: 1.82 s, total: 2.21 s
Wall time: 18 s


In [28]:
%%time
for k,v in POSES.items():
    print(k)
    poses = v["merge_poses"]
    compounds = v["merge_compounds"]
    elabs = compounds.elabs
    POSES[k]["merge_smiles"] = compounds.smiles

{C × 3234}

In [38]:
compounds = POSES[TAGS[1]]["merge_compounds"]
compounds

{C × 11534}

In [39]:
elabs = compounds.elabs
elabs

{C × 20993}

In [40]:
elabs.bases

{C × 7}

In [67]:
%%time
dfs = []
for k,v in POSES.items():
    print(k)
    compounds = v["merge_compounds"]
    elabs = compounds.elabs
    print(elabs)
    df = elabs.get_df(alias=False, bases=True, debug=True)
    print(len(df))
    df["site"] = k
    df.rename(columns={"bases":"scaffold"}, inplace=True)
    scaffolds = elabs.bases
    print("scaffolds", scaffolds)
    
    lookup = animal.db.get_compound_id_smiles_dict(scaffolds)
    df["scaffold"] = df["scaffold"].apply(lambda x: lookup[list(x)[0]])

    sql = f"""
    SELECT p1.pose_compound, p2.pose_path, MIN(p1.pose_distance_score) FROM pose as p1
    INNER JOIN pose AS p2 ON p1.pose_reference = p2.pose_id
    WHERE p1.pose_compound IN {elabs.str_ids}
    GROUP BY p1.pose_compound
    """

    records = animal.db.execute(sql)

    lookup = { c:path for c,path,_ in records }

    df["protein"] = df.apply(lambda row: lookup.get(row.name, None), axis=1)
    
    dfs.append(df)
df = pd.concat(dfs)
df["protein"] = df["protein"].apply(lambda x: Path(x).name if x else None)
df.head()

Green Site/RNA tunnel

{C × 148292}

DEBUG: querying...

Output()

DEBUG: adding scaffold column(s)

...

DEBUG: adding scaffold column(s)

3234

scaffolds {C × 1}

CPU times: user 3.23 s, sys: 4.76 s, total: 7.99 s
Wall time: 41.4 s


,smiles,scaffold,site,reference_path
id,,,,
241786,BrCC(NC1CCCc2c(Br)ccnc21)c1cccnc1N1CCCC1,c1cnc2c(c1)CCCC2NCc1cccnc1N1CCCC1,Green Site/RNA tunnel,None
241784,BrCC(NC1CCCc2cc(Br)cnc21)c1cccnc1N1CCCC1,c1cnc2c(c1)CCCC2NCc1cccnc1N1CCCC1,Green Site/RNA tunnel,None
241785,BrCC(NC1CCCc2ccc(Br)nc21)c1cccnc1N1CCCC1,c1cnc2c(c1)CCCC2NCc1cccnc1N1CCCC1,Green Site/RNA tunnel,None
240771,BrCC(NC1CCCc2cccnc21)c1cccnc1N1CCCC1,c1cnc2c(c1)CCCC2NCc1cccnc1N1CCCC1,Green Site/RNA tunnel,None
240575,BrCC(c1cccnc1N1CCCC1)N1C2CCC1c1ncccc12,c1cnc2c(c1)CCCC2NCc1cccnc1N1CCCC1,Green Site/RNA tunnel,None


In [77]:
df

,smiles,scaffold,site,protein
id,,,,
241786,BrCC(NC1CCCc2c(Br)ccnc21)c1cccnc1N1CCCC1,c1cnc2c(c1)CCCC2NCc1cccnc1N1CCCC1,Green Site/RNA tunnel,None
241784,BrCC(NC1CCCc2cc(Br)cnc21)c1cccnc1N1CCCC1,c1cnc2c(c1)CCCC2NCc1cccnc1N1CCCC1,Green Site/RNA tunnel,None
241785,BrCC(NC1CCCc2ccc(Br)nc21)c1cccnc1N1CCCC1,c1cnc2c(c1)CCCC2NCc1cccnc1N1CCCC1,Green Site/RNA tunnel,None
240771,BrCC(NC1CCCc2cccnc21)c1cccnc1N1CCCC1,c1cnc2c(c1)CCCC2NCc1cccnc1N1CCCC1,Green Site/RNA tunnel,None
240575,BrCC(c1cccnc1N1CCCC1)N1C2CCC1c1ncccc12,c1cnc2c(c1)CCCC2NCc1cccnc1N1CCCC1,Green Site/RNA tunnel,None
...,...,...,...,...
77222,CCS(=O)(=O)N(C)Cc1ccc(C(=O)N2C(CC3CC3)C(=O)N2C...,CCS(=O)(=O)NCc1ccc(C(=O)N2CC(=O)N2)cc1,Thumb Site 2,d0174a.pdb
77223,CCS(=O)(=O)N(C)Cc1ccc(C(=O)N2C(CC(F)F)C(=O)N2C...,CCS(=O)(=O)NCc1ccc(C(=O)N2CC(=O)N2)cc1,Thumb Site 2,d0174a.pdb
77224,CCS(=O)(=O)N(C)Cc1ccc(C(=O)N2N(C3CC3)C(=O)C2(C...,CCS(=O)(=O)NCc1ccc(C(=O)N2CC(=O)N2)cc1,Thumb Site 2,d0174a.pdb


In [78]:
df.to_csv("Flavi_RdRp_elabs.csv")

In [59]:
elabs = animal.compounds[list(df.index.values)]
elabs

{C × 148292}

In [60]:
best_poses = elabs.best_placed_poses
best_poses

{P × 75994}

In [62]:
records = animal.db.select_where(table="pose", query="pose_compound, pose_path", key=f"pose_id IN {best_poses.str_ids}", multiple=True)
pose_info = {}
for compound_id, pose_path in records:
    pose_info[compound_id] = pose_path

In [63]:
pose_info

{55856: '/opt/xchem-fragalysis-2/maxwin/BulkDock/SCRATCH/217082/C55856-P102/C55856-P102.minimised.mol',
 28177: '/opt/xchem-fragalysis-2/maxwin/BulkDock/SCRATCH/220455/C28177-P104/C28177-P104.minimised.mol',
 60764: '/opt/xchem-fragalysis-2/maxwin/BulkDock/SCRATCH/220457/C60764-P218/C60764-P218.minimised.mol',
 70938: '/opt/xchem-fragalysis-2/maxwin/RdRp_FFF/syndirella/feb_elabs/elabs_84/KZGPDDSEMZONAM-UHFFFAOYSA-N/output/KZGPDDSEMZONAM-UHFFFAOYSA-N-DX3HCc-8-B/KZGPDDSEMZONAM-UHFFFAOYSA-N-DX3HCc-8-B.minimised.mol',
 70942: '/opt/xchem-fragalysis-2/maxwin/RdRp_FFF/syndirella/feb_elabs/elabs_84/KZGPDDSEMZONAM-UHFFFAOYSA-N/output/KZGPDDSEMZONAM-UHFFFAOYSA-N-DX3HCc-12-A/KZGPDDSEMZONAM-UHFFFAOYSA-N-DX3HCc-12-A.minimised.mol',
 70944: '/opt/xchem-fragalysis-2/maxwin/RdRp_FFF/syndirella/feb_elabs/elabs_84/KZGPDDSEMZONAM-UHFFFAOYSA-N/output/KZGPDDSEMZONAM-UHFFFAOYSA-N-DX3HCc-14-B/KZGPDDSEMZONAM-UHFFFAOYSA-N-DX3HCc-14-B.minimised.mol',
 70946: '/opt/xchem-fragalysis-2/maxwin/RdRp_FFF/syndirella/

In [64]:
import tarfile
with tarfile.open("Flavi_RdRp_elab_poses.tar.gz", "w:gz") as tar:
    for c_id, actual_path in mrich.track(pose_info.items()):
        tar.add(actual_path, arcname=f"{c_id}.mol")

Output()

In [81]:
import tarfile
with tarfile.open("Flavi_RdRp_elab_proteins.tar.gz", "w:gz") as tar:
    for name in set(p for p in df["protein"].values if p):
        p = Path("../../BulkDock/TARGETS/Flavi_NS5_RdRp") / "aligned_files" / name.removesuffix(".pdb") / name
        print(name, p, p.exists())
        tar.add(p, arcname=name)

d0472b.pdb ../../BulkDock/TARGETS/Flavi_NS5_RdRp/aligned_files/d0472b/d0472b.pdb True

z0637a.pdb ../../BulkDock/TARGETS/Flavi_NS5_RdRp/aligned_files/z0637a/z0637a.pdb True

z0364a.pdb ../../BulkDock/TARGETS/Flavi_NS5_RdRp/aligned_files/z0364a/z0364a.pdb True

d0160a.pdb ../../BulkDock/TARGETS/Flavi_NS5_RdRp/aligned_files/d0160a/d0160a.pdb True

z0041a.pdb ../../BulkDock/TARGETS/Flavi_NS5_RdRp/aligned_files/z0041a/z0041a.pdb True

...

z0111a.pdb ../../BulkDock/TARGETS/Flavi_NS5_RdRp/aligned_files/z0111a/z0111a.pdb True

z0221a.pdb ../../BulkDock/TARGETS/Flavi_NS5_RdRp/aligned_files/z0221a/z0221a.pdb True

d0264a.pdb ../../BulkDock/TARGETS/Flavi_NS5_RdRp/aligned_files/d0264a/d0264a.pdb True

z0136a.pdb ../../BulkDock/TARGETS/Flavi_NS5_RdRp/aligned_files/z0136a/z0136a.pdb True

z0767a.pdb ../../BulkDock/TARGETS/Flavi_NS5_RdRp/aligned_files/z0767a/z0767a.pdb True